In [1]:
import tensorflow as tf
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### shapes 

In [2]:
g = tf.Graph()

with g.as_default():
    ## define tensors t1, t2, t3
    t1 = tf.constant(np.pi)
    t2 = tf.constant([1, 2, 3, 4])
    t3 = tf.constant([[1, 2], [3, 4]])
    
    # ranks
    r1 = tf.rank(t1)
    r2 = tf.rank(t2)
    r3 = tf.rank(t3)
    
    
    # shapes
    s1 = t1.get_shape()
    s2 = t2.get_shape()
    s3 = t3.get_shape()
    
    print('Shapes:', s1, s2, s3)

Shapes: () (4,) (2, 2)


In [4]:
with tf.Session(graph=g) as sess:
    print('Ranks:',
    r1.eval(),
    r2.eval(),
    r3.eval())

Ranks: 0 1 2


### variables

In [1]:
import tensorflow as tf
import numpy as np

g1 = tf.Graph()

with g1.as_default():
    w = tf.Variable(np.array([[1,2,3,4],
                              [5,6,7,8]]), name='w')
    print(w)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<tf.Variable 'w:0' shape=(2, 4) dtype=int64_ref>


In [2]:
# note empty until initialized
with tf.Session(graph=g1) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))

[[1 2 3 4]
 [5 6 7 8]]


In [3]:
# scoping

import tensorflow as tf

g = tf.Graph()

with g.as_default():
    with tf.variable_scope('net_A'):
        with tf.variable_scope('layer-1'):
            w1 = tf.Variable(tf.random_normal(
            shape=(10,4)), name='weights')
    with tf.variable_scope('layer-2'):
        w2 = tf.Variable(tf.random_normal(
        shape=(20,10)), name='weights')
    with tf.variable_scope('net_B'):
        with tf.variable_scope('layer-1'):
            w3 = tf.Variable(tf.random_normal(
            shape=(10,4)), name='weights')

    print(w1)
    print(w2)
    print(w3)

<tf.Variable 'net_A/layer-1/weights:0' shape=(10, 4) dtype=float32_ref>
<tf.Variable 'layer-2/weights:0' shape=(20, 10) dtype=float32_ref>
<tf.Variable 'net_B/layer-1/weights:0' shape=(10, 4) dtype=float32_ref>


In [5]:
def build_classifier(data, labels, n_classes=2):
    data_shape = data.get_shape().as_list()
    weights = tf.get_variable(name='weights',
                             shape=(data_shape[1], n_classes),
                             dtype=tf.float32)
    bias = tf.get_variable(name='bias',
                          initializer=tf.zeros(shape=n_classes))
    logits = tf.add(tf.matmul(data, weights),
                   bias,
                   name='logits')
    
    return logits, tf.nn.softmax(logits)



def build_generator(data, n_hidden):
    data_shape = data.get_shape().as_list()
    w1 = tf.Variable(tf.random_normal(shape=(data_shape[1], n_hidden)),
                    name='w1')
    b1 = tf.Variable(tf.zeros(shape=n_hidden), name='b1')
    hidden = tf.add(tf.matmul(data, w1),
                   b1,
                   name='hidden_preact')
    hidden = tf.nn.relu(hidden, 'hidden_activation')
    
    w2 = tf.Variable(tf.random_normal(shape=(n_hidden, data_shape[1])),
                    name='w2')
    b2 = tf.Variable(tf.zeros(shape=data_shape[1]),
                    name='b2')
    
    output = tf.add(tf.matmul(hidden, w2),
                   b2,
                   name='output')
    
    return output, tf.nn.sigmoid(output)

#
### graph
#

batch_size=64
g = tf.Graph()

with g.as_default():
    tf_X = tf.placeholder(shape=(batch_size, 100),
                         dtype=tf.float32,
                         name='tf_X')
    
    with tf.variable_scope('generator'):
        gen_out1 = build_generator(data=tf_X,
                                  n_hidden=50)
    
    with tf.variable_scope('classifier') as scope:
        cls_out1 = build_classifier(data=tf_X,
                                   labels=tf.ones(shape=batch_size))
        
        scope.reuse_variables()
        cls_out2 = build_classifier(data=gen_out1[1],
                                   labels=tf.zeros(shape=batch_size))